# Evaluation Metrics and Visualization

This notebook provides a comprehensive guide to evaluating seismic reconstruction quality.

**Prerequisites:**
- Python 3.10+
- Basic understanding of signal quality metrics

**Estimated Runtime:** 5 minutes

**Metrics Covered:**
- Signal-to-Noise Ratio (SNR)
- Mean Squared Error (MSE)
- Peak Signal-to-Noise Ratio (PSNR)
- Structural Similarity Index (SSIM)
- Frequency Domain Correlation
- Phase Coherence

In [ ]:
# Uncomment to install:
# !pip install promethium-seismic==1.0.3

In [ ]:
import promethium
from promethium import (
    signal_to_noise_ratio,
    mean_squared_error,
    peak_signal_to_noise_ratio,
    structural_similarity_index,
    frequency_domain_correlation,
    phase_coherence,
    evaluate_reconstruction,
    generate_synthetic_traces,
    add_noise,
    set_seed,
)

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

print(f"Promethium version: {promethium.__version__}")
set_seed(42)

## 1. Create Test Scenarios

In [ ]:
# Generate clean data
clean_data, metadata = generate_synthetic_traces(
    n_traces=50,
    n_samples=500,
    sample_rate=250.0,
    seed=42
)

# Create multiple degradation scenarios
scenarios = {
    'low_noise': add_noise(clean_data, noise_level=0.1, seed=42),
    'medium_noise': add_noise(clean_data, noise_level=0.3, seed=42),
    'high_noise': add_noise(clean_data, noise_level=0.5, seed=42),
}

# Simulate reconstructions (with decreasing quality)
from scipy.ndimage import gaussian_filter1d
reconstructions = {
    'good_recovery': np.array([gaussian_filter1d(t, sigma=1) for t in scenarios['medium_noise']]),
    'moderate_recovery': np.array([gaussian_filter1d(t, sigma=3) for t in scenarios['medium_noise']]),
    'poor_recovery': np.array([gaussian_filter1d(t, sigma=6) for t in scenarios['medium_noise']]),
}

print(f"Created {len(scenarios)} noise scenarios and {len(reconstructions)} reconstruction scenarios")

## 2. Individual Metrics

In [ ]:
# Compute individual metrics for demonstration
y_true = clean_data[0:1]  # Single trace
y_pred = reconstructions['good_recovery'][0:1]

print("Individual Metric Examples")
print("=" * 50)

# SNR
snr = signal_to_noise_ratio(y_true, y_pred)
print(f"SNR: {snr:.2f} dB")
print("  Higher is better. Measures signal power vs noise power.")
print()

# MSE
mse = mean_squared_error(y_true, y_pred)
print(f"MSE: {mse:.6f}")
print("  Lower is better. Average squared difference.")
print()

# PSNR
psnr = peak_signal_to_noise_ratio(y_true, y_pred)
print(f"PSNR: {psnr:.2f} dB")
print("  Higher is better. Peak signal vs noise ratio.")
print()

# SSIM
ssim = structural_similarity_index(y_true, y_pred)
print(f"SSIM: {ssim:.4f}")
print("  Range [-1, 1]. 1 = perfect similarity.")
print()

# Frequency Domain Correlation
freq_corr = frequency_domain_correlation(y_true, y_pred)
print(f"Frequency Correlation: {freq_corr:.4f}")
print("  Range [-1, 1]. 1 = perfect frequency match.")
print()

# Phase Coherence
phase_coh = phase_coherence(y_true, y_pred)
print(f"Phase Coherence: {phase_coh:.4f}")
print("  Range [0, 1]. 1 = perfect phase preservation.")

## 3. Comprehensive Evaluation

In [ ]:
# Evaluate all reconstruction scenarios
results = []

for name, recon in reconstructions.items():
    metrics = evaluate_reconstruction(clean_data, recon)
    metrics['scenario'] = name
    results.append(metrics)

# Create DataFrame for easy comparison
df = pd.DataFrame(results)
df = df.set_index('scenario')

print("Reconstruction Quality Comparison")
print("=" * 60)
print(df.round(4).to_string())

In [ ]:
# Visualize metrics comparison
fig, axes = plt.subplots(2, 3, figsize=(14, 8))
metrics_to_plot = ['snr', 'mse', 'psnr', 'ssim', 'freq_correlation', 'phase_coherence']

for ax, metric in zip(axes.flatten(), metrics_to_plot):
    values = df[metric].values
    bars = ax.bar(df.index, values, color=['green', 'orange', 'red'])
    ax.set_title(metric.upper().replace('_', ' '))
    ax.set_ylabel('Value')
    ax.tick_params(axis='x', rotation=45)
    ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 4. Visual Comparison

In [ ]:
# Visual comparison of reconstruction quality
trace_idx = 10
t = np.arange(metadata['n_samples']) / metadata['sample_rate']

fig, axes = plt.subplots(5, 1, figsize=(14, 12), sharex=True)

# Original
axes[0].plot(t, clean_data[trace_idx], 'b-', linewidth=0.8)
axes[0].set_ylabel('Amplitude')
axes[0].set_title('Original Clean Signal')
axes[0].grid(True, alpha=0.3)

# Noisy input
axes[1].plot(t, scenarios['medium_noise'][trace_idx], 'r-', linewidth=0.8)
axes[1].set_ylabel('Amplitude')
axes[1].set_title('Noisy Input (30% noise)')
axes[1].grid(True, alpha=0.3)

# Reconstructions
colors = ['green', 'orange', 'brown']
for i, (name, recon) in enumerate(reconstructions.items()):
    ax = axes[i + 2]
    ax.plot(t, recon[trace_idx], color=colors[i], linewidth=0.8)
    snr_val = df.loc[name, 'snr']
    ax.set_ylabel('Amplitude')
    ax.set_title(f'{name.replace("_", " ").title()} (SNR: {snr_val:.1f} dB)')
    ax.grid(True, alpha=0.3)

axes[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()

## 5. Error Maps

In [ ]:
# Compute and visualize error maps
fig, axes = plt.subplots(1, 3, figsize=(14, 5))

for i, (name, recon) in enumerate(reconstructions.items()):
    error = clean_data - recon
    
    clip = np.percentile(np.abs(error), 99)
    im = axes[i].imshow(
        error.T[:200, :],  # Show first 200 time samples
        aspect='auto',
        cmap='coolwarm',
        vmin=-clip,
        vmax=clip
    )
    axes[i].set_xlabel('Trace')
    axes[i].set_ylabel('Time Sample')
    axes[i].set_title(f'Error Map: {name.replace("_", " ").title()}')

plt.colorbar(im, ax=axes, label='Error Amplitude', shrink=0.6)
plt.tight_layout()
plt.show()

## 6. Summary

This notebook demonstrated:

1. **Individual Metrics**: SNR, MSE, PSNR, SSIM, frequency correlation, phase coherence
2. **Comprehensive Evaluation**: Using `evaluate_reconstruction()` for all metrics
3. **Comparative Analysis**: Tabular and visual comparison of scenarios
4. **Error Visualization**: Error maps for spatial analysis

### Metric Selection Guidelines

| Use Case | Recommended Metrics |
|----------|--------------------|
| General quality | SNR, PSNR |
| Perceptual similarity | SSIM |
| Frequency preservation | Frequency Correlation |
| Phase-sensitive applications | Phase Coherence |
| Optimization target | MSE, PSNR |